In [ ]:

!pip install selenium pandas openpyxl webdriver-manager


  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.7 MB 11.2 MB/s eta 0:00:01
   ---------------------- ----------------- 5.5/9.7 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.7 MB 15.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 15.3 MB/s eta 0:00:00
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)

  Attempting uninstall: urllib3

    Found existing installation: urllib3 2.3.0

    Uninstalling urllib3-2.3.0:

      Successfully uninstalled urllib3-2.3.0

   ----- ---------------------------------- 1/8 [urllib3]
   ----- ---------------------------------- 1/8 [urllib3]
  Attempting uninstall: typing_extensions
   ----- ---------------------------------- 1/8 [urllib3]
    Found existing installation: typing_extensions 4.12.2
   ----- ---------------------------------- 1/8 [u

# main 크롤링 - 전체 - 정상실행

In [ ]:
"""
Olive Young Global - Web Crawler (Auto Country Selection)
K-Beauty 제품 및 리뷰 데이터 수집 크롤러

수집 데이터: 제품 정보, 가격, 평점, 성분, 리뷰(최신 50개)
대상 시장: USA, Japan, China (수동 선택)
카테고리: 9개 (Skincare 2개, Makeup 4개, Hair 1개, Mask 1개, Suncare 1개)
"""

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from datetime import datetime
import os
import json


class OliveYoungCrawlerAuto:

    """
    올리브영 글로벌 웹사이트 크롤러 (자동 국가 선택)

    Features:
    - 자동 국가 선택 (USA, Japan, China)
    - 제품 상세 정보 수집 (제품명, 브랜드, 가격, 평점, 성분, 설명)
    - 리뷰 데이터 수집 (내용, 날짜, 평점, 카테고리별 세부평점)
    - 실시간 저장 (5개 제품마다)
    - View 36 자동 설정
    """

    def __init__(self, country='Japan', save_folder='./oliveyoung_data'):
        """
        크롤러 초기화

        Args:
            country (str): 국가 선택 ('USA', 'Japan', 'China')
            save_folder (str): 저장 폴더 경로
        """
        from selenium.webdriver.chrome.service import Service
        from webdriver_manager.chrome import ChromeDriverManager

        # 국가 설정
        country_map = {
            'USA': ('USA', '미국', 'USA'),
            'Japan': ('Japan', '일본', 'JAPAN'),
            'China': ('China', '중국', 'MAINLAND, CHINA')
        }

        if country not in country_map:
            raise ValueError("country는 'USA', 'Japan', 'China' 중 하나여야 합니다.")

        self.country_code, self.country_name_kr, self.country_name_en = country_map[country]

        print("=" * 70)
        print(f"Olive Young Global Crawler - {self.country_code}")
        print("=" * 70)

        # 저장 폴더 설정
        self.save_folder = save_folder
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        print(f"[INFO] 저장 폴더: {os.path.abspath(save_folder)}")

        # Chrome 드라이버 설정
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, 10)

        # 데이터 저장 리스트
        self.all_data = []

        # 파일명 생성
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.excel_filename = os.path.join(save_folder, f'oliveyoung_{self.country_name_kr}_{timestamp}.xlsx')

        # 국가 자동 선택
        self.select_country()

    def select_country(self):
        """
        올리브영 사이트에서 국가 자동 선택
        """
        print(f"\n[STEP 0] 국가 선택: {self.country_name_kr}")

        try:
            # 메인 페이지 접속
            self.driver.get("https://global.oliveyoung.com")
            time.sleep(5)  # 충분히 대기

            # 페이지 스크롤 (요소가 화면에 보이도록)
            self.driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)

            # 국가 선택 버튼 클릭
            try:
                # 여러 방법으로 버튼 찾기
                country_button = None

                # 방법 1: ID로 찾기
                try:
                    country_button = self.wait.until(
                        EC.presence_of_element_located((By.ID, 'sel_headDivCntry'))
                    )
                    print("[DEBUG] ID로 버튼 찾음")
                except:
                    pass

                # 방법 2: data-testid로 찾기
                if not country_button:
                    try:
                        country_button = self.driver.find_element(
                            By.CSS_SELECTOR, '[data-testid="header-country-change-select-country-button"]'
                        )
                        print("[DEBUG] data-testid로 버튼 찾음")
                    except:
                        pass

                # 방법 3: XPath로 찾기 (현재 국가 텍스트 포함)
                if not country_button:
                    try:
                        country_button = self.driver.find_element(
                            By.XPATH, "//div[@id='sel_headDivCntry' or contains(@class, 'selectbox-trigger')]"
                        )
                        print("[DEBUG] XPath로 버튼 찾음")
                    except:
                        pass

                if not country_button:
                    print("[WARNING] 국가 선택 버튼을 찾을 수 없습니다.")
                    print("[INFO] 수동으로 국가를 선택하거나 기본 국가로 진행합니다.")
                    time.sleep(10)  # 수동 선택 시간 제공
                    return

                # JavaScript로 강제 클릭
                print("[INFO] 국가 선택 버튼 클릭 중...")
                self.driver.execute_script("arguments[0].scrollIntoView(true);", country_button)
                time.sleep(1)
                self.driver.execute_script("arguments[0].click();", country_button)
                time.sleep(3)
                print("[SUCCESS] 국가 선택 팝업 열기 완료")

                # 드롭다운에서 국가 선택
                print(f"[INFO] {self.country_name_kr} 선택 중...")
                country_xpath = f"//span[contains(text(), '{self.country_name_en}')]"

                try:
                    country_option = self.wait.until(
                        EC.element_to_be_clickable((By.XPATH, country_xpath))
                    )
                    self.driver.execute_script("arguments[0].click();", country_option)
                    time.sleep(2)
                    print(f"[SUCCESS] {self.country_name_kr} 선택 완료")
                except:
                    print(f"[WARNING] {self.country_name_kr} 옵션을 찾을 수 없습니다.")
                    print("[INFO] 수동으로 국가를 선택해주세요.")
                    time.sleep(10)
                    return

                # Save 버튼 클릭
                try:
                    save_button = self.wait.until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Save') or contains(text(), '저장')]"))
                    )
                    self.driver.execute_script("arguments[0].click();", save_button)
                    time.sleep(3)
                    print("[SUCCESS] 국가 설정 저장 완료")
                except:
                    print("[WARNING] Save 버튼을 찾을 수 없습니다.")
                    print("[INFO] 엔터를 눌러 진행하거나 수동으로 저장해주세요.")
                    time.sleep(5)

            except Exception as e:
                print(f"[WARNING] 국가 선택 중 오류: {str(e)[:100]}")
                print("[INFO] 수동으로 국가를 선택하거나 기본 국가로 진행합니다.")
                time.sleep(10)  # 수동 선택 시간

        except Exception as e:
            print(f"[ERROR] 국가 선택 실패: {e}")
            print("[INFO] 기본 설정으로 계속 진행합니다.")
            time.sleep(5)

    def get_product_urls(self, category_url, limit=30):
        """
        카테고리 페이지에서 제품 URL 수집

        Args:
            category_url (str): 카테고리 페이지 URL
            limit (int): 수집할 제품 개수 (기본: 30)

        Returns:
            list: 제품 상세 페이지 URL 리스트
        """
        print(f"\n{'='*70}")
        print(f"[STEP 1] 카테고리 페이지 접속 중...")
        print(f"{'='*70}")

        try:
            # View 36 파라미터 추가
            if '?' in category_url:
                url_with_view = f"{category_url}&pageSize=36"
            else:
                url_with_view = f"{category_url}?pageSize=36"

            self.driver.get(url_with_view)
            time.sleep(5)

            # View 36 버튼 클릭 시도
            try:
                view_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'button, a')
                for btn in view_buttons:
                    if '36' in btn.text:
                        btn.click()
                        time.sleep(2)
                        print("[SUCCESS] View 36 설정 완료")
                        break
            except:
                pass

            # 페이지 스크롤
            for i in range(3):
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.5)

            # 제품 링크 수집
            product_links = self.driver.find_elements(By.CSS_SELECTOR, 'a[href*="/product/detail"]')

            urls = []
            for link in product_links:
                href = link.get_attribute('href')
                if href and href not in urls:
                    urls.append(href)

            print(f"[SUCCESS] 제품 URL 수집 완료: {len(urls)}개")
            return urls[:limit]

        except Exception as e:
            print(f"[ERROR] URL 수집 실패: {e}")
            return []

    def extract_product_and_reviews(self, product_url, rank, major_category, sub_category):
        """
        제품 상세 정보 및 리뷰 수집

        Args:
            product_url (str): 제품 상세 페이지 URL
            rank (int): 카테고리 내 순위
            major_category (str): 대분류 카테고리
            sub_category (str): 중분류 카테고리

        Returns:
            list: 제품 정보 + 리뷰가 결합된 데이터 리스트
        """
        all_rows = []

        try:
            self.driver.get(product_url)
            time.sleep(3)

            # === 제품 기본 정보 수집 ===
            product_info = {
                'country': self.country_code,
                'major_category': major_category,
                'sub_category': sub_category,
                'rank': rank,
                'product_url': product_url
            }

            # 제품명
            try:
                product_info['product_name'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'dt[data-testid="product-name"]'
                ).text.strip()
            except:
                product_info['product_name'] = ''

            # 브랜드
            try:
                brand_elem = self.driver.find_element(By.CSS_SELECTOR, 'dt.notranslate')
                product_info['brand'] = brand_elem.text.strip()
            except:
                product_info['brand'] = ''

            # 할인율
            try:
                discount = self.driver.find_element(By.CSS_SELECTOR, 'span.discount-rate').text.strip()
                product_info['discount_rate'] = discount
            except:
                product_info['discount_rate'] = ''

            # 가격 (원가)
            try:
                price_elem = self.driver.find_element(By.CSS_SELECTOR, 'dt.price')
                price_spans = price_elem.find_elements(By.CSS_SELECTOR, 'div > span')
                if price_spans:
                    product_info['price'] = price_spans[0].text.strip()
                else:
                    product_info['price'] = ''
            except:
                product_info['price'] = ''

            # 평균 평점
            try:
                rating_elem = self.driver.find_element(By.CSS_SELECTOR, 'dl.prd-rating-info')
                rating_spans = rating_elem.find_elements(By.TAG_NAME, 'span')
                for span in rating_spans:
                    text = span.text.strip()
                    if text and text[0].isdigit():
                        product_info['rating'] = text
                        break
                if 'rating' not in product_info:
                    product_info['rating'] = ''
            except:
                product_info['rating'] = ''

            # 성분 (버튼 클릭 후 수집)
            try:
                ingr_button = self.driver.find_element(
                    By.CSS_SELECTOR, 'a[data-testid="product-featuredingredients-link"]'
                )
                ingr_button.click()
                time.sleep(2)

                product_info['ingredients'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'div[data-testid="product-featuredingredients-content"]'
                ).text.strip()
            except:
                product_info['ingredients'] = ''

            # 제품 설명 (버튼 클릭 후 수집)
            try:
                desc_button = self.driver.find_element(
                    By.CSS_SELECTOR, 'a[data-testid="product-whyweloveit-link"]'
                )
                desc_button.click()
                time.sleep(2)

                product_info['description'] = self.driver.find_element(
                    By.CSS_SELECTOR, 'div[data-testid="product-whyweloveit-content"]'
                ).text.strip()
            except:
                product_info['description'] = ''

            print(f"  [PRODUCT {rank:2d}] {product_info['brand']:20s} | {product_info['product_name'][:50]}")
            print(f"               가격: {product_info['price']} (할인: {product_info['discount_rate']}) | 평점: {product_info['rating']}")

            # === 리뷰 데이터 수집 ===
            print(f"    [STEP 2] 리뷰 수집 시작...")

            # 리뷰 섹션으로 스크롤
            time.sleep(2)
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(2)

            # 최신순 정렬
            try:
                newest_buttons = self.driver.find_elements(By.CSS_SELECTOR, 'li[tabindex="0"]')
                for button in newest_buttons:
                    if 'Newest' in button.text or 'newest' in button.text:
                        button.click()
                        time.sleep(2)
                        print(f"    [SUCCESS] 최신순 정렬 완료")
                        break
            except:
                print(f"    [WARNING] 최신순 버튼 찾기 실패 (기본 정렬 사용)")

            # More 버튼 반복 클릭 (리뷰 50개 로드)
            more_clicks = 0
            while more_clicks < 5:
                try:
                    more_btn = self.driver.find_element(By.CSS_SELECTOR, 'button.review-list-more-btn')
                    if more_btn.is_displayed():
                        self.driver.execute_script("arguments[0].scrollIntoView(true);", more_btn)
                        time.sleep(1)
                        more_btn.click()
                        more_clicks += 1
                        print(f"    [LOADING] More 버튼 클릭 ({more_clicks}/5)")
                        time.sleep(2)

                        # 리뷰 개수 확인
                        reviews = self.driver.find_elements(By.CSS_SELECTOR, 'div.product-review-unit-main')
                        if len(reviews) >= 50:
                            print(f"    [SUCCESS] 목표 달성: {len(reviews)}개 리뷰 로드 완료")
                            break
                    else:
                        break
                except NoSuchElementException:
                    print(f"    [INFO] More 버튼 없음 (모든 리뷰 로드 완료)")
                    break
                except:
                    break

            # 리뷰 파싱
            review_elements = self.driver.find_elements(By.CSS_SELECTOR, 'div.product-review-unit-main')
            print(f"    [INFO] 발견된 리뷰: {len(review_elements)}개")

            for idx, review_elem in enumerate(review_elements[:50], 1):
                try:
                    row = product_info.copy()

                    # 리뷰 전체 별점 (1-5점)
                    try:
                        filled_stars = review_elem.find_elements(
                            By.CSS_SELECTOR, 'div.product-review-unit-header div.review-star-rating div.icon-star.filled'
                        )
                        row['review_rating'] = len(filled_stars) / 2 if filled_stars else ''
                    except:
                        row['review_rating'] = ''

                    # 리뷰 내용
                    try:
                        row['review_content'] = review_elem.find_element(
                            By.CSS_SELECTOR, 'div.review-unit-cont-comment'
                        ).text.strip()
                    except:
                        row['review_content'] = ''

                    # 작성 날짜
                    try:
                        parent = review_elem.find_element(By.XPATH, '..')
                        date_elem = parent.find_element(
                            By.CSS_SELECTOR, 'span.review-write-info-date.notranslate'
                        )
                        row['review_date'] = date_elem.get_attribute('textContent').strip()
                    except:
                        row['review_date'] = ''

                    # 카테고리별 세부 평점
                    try:
                        detail_ratings = {}
                        detail_items = review_elem.find_elements(By.CSS_SELECTOR, 'ul.list-review-evlt > li')

                        for item in detail_items:
                            try:
                                category_name = item.find_element(By.TAG_NAME, 'span').text.strip()
                                filled = item.find_elements(By.CSS_SELECTOR, 'div.icon-star.filled')
                                rating_value = len(filled) / 2

                                if category_name:
                                    detail_ratings[category_name] = rating_value
                            except:
                                continue

                        row['review_detail_ratings'] = json.dumps(detail_ratings) if detail_ratings else ''
                    except:
                        row['review_detail_ratings'] = ''

                    all_rows.append(row)

                    if idx % 20 == 0:
                        print(f"    [PROGRESS] 리뷰 수집 중... {idx}/{min(50, len(review_elements))}")

                except:
                    continue

            print(f"    [SUCCESS] 리뷰 수집 완료: {len(all_rows)}개")
            return all_rows

        except Exception as e:
            print(f"  [ERROR] 제품 처리 실패: {str(e)[:50]}")
            return all_rows

    def save_to_excel(self):
        """수집된 데이터를 Excel 파일로 저장"""
        if not self.all_data:
            print("[WARNING] 저장할 데이터가 없습니다.")
            return

        df = pd.DataFrame(self.all_data)

        # 컬럼 순서 정렬 (15개)
        columns = [
            'country', 'major_category', 'sub_category', 'rank',
            'product_name', 'brand', 'discount_rate', 'price', 'rating',
            'ingredients', 'description',
            'review_content', 'review_date', 'review_rating', 'review_detail_ratings',
            'product_url'
        ]

        existing_cols = [c for c in columns if c in df.columns]
        df = df[existing_cols]

        df.to_excel(self.excel_filename, index=False, engine='openpyxl')
        print(f"\n[SAVED] {self.excel_filename} ({len(df)}행)")

    def crawl_all_categories(self):
        """
        전체 카테고리 크롤링 (9개 카테고리)
        """
        # 9개 카테고리 정의
        categories = {
            'Skincare': {
                'Moisturizers': '1000000009',
                'Cleansers': '1000000010'
            },
            'Makeup': {
                'Face': '1000000032',
                'Eye': '1000000040',
                'Lip': '1000000045',
                'Nail': '1000000049'
            },
            'Hair': {
                'All Hair': '1000000070'
            },
            'Mask': {
                'All Face Masks': '1000000003'
            },
            'Suncare': {
                'All Suncare': '1000000011'
            }
        }

        total_categories = sum(len(subs) for subs in categories.values())
        current = 0

        print(f"\n{'='*70}")
        print(f"[START] 전체 크롤링 시작 ({self.country_name_kr})")
        print(f"[INFO] 총 {total_categories}개 카테고리")
        print(f"[INFO] 예상 소요 시간: 약 {total_categories * 15}분")
        print(f"{'='*70}")

        overall_start = time.time()

        # 각 카테고리 크롤링
        for major_category, sub_categories in categories.items():
            for sub_category, category_id in sub_categories.items():
                current += 1

                print(f"\n{'='*70}")
                print(f"[PROGRESS] 카테고리 [{current}/{total_categories}]")
                print(f"[CATEGORY] {major_category} > {sub_category}")
                print(f"{'='*70}")

                category_url = f"https://global.oliveyoung.com/display/category?ctgrNo={category_id}"

                # 카테고리 크롤링
                self.crawl_category(category_url, major_category, sub_category, limit=30)

                print(f"\n[INFO] 현재까지 수집된 데이터: {len(self.all_data)}행")

        # 전체 완료
        total_elapsed = time.time() - overall_start
        print(f"\n{'='*70}")
        print(f"[COMPLETED] 전체 크롤링 완료!")
        print(f"[TIME] 총 소요 시간: {total_elapsed/60:.1f}분")
        print(f"[RESULT] 최종 수집 데이터: {len(self.all_data)}행")
        print(f"{'='*70}")

    def crawl_category(self, category_url, major_category, sub_category, limit=30):
        """
        카테고리 전체 크롤링 실행

        Args:
            category_url (str): 카테고리 페이지 URL
            major_category (str): 대분류 카테고리명
            sub_category (str): 중분류 카테고리명
            limit (int): 수집할 제품 개수
        """
        print(f"\n{'='*70}")
        print(f"[CATEGORY] {major_category} > {sub_category}")
        print(f"{'='*70}")

        start_time = time.time()

        # 1. 제품 URL 수집
        urls = self.get_product_urls(category_url, limit)

        if not urls:
            print("[ERROR] 제품 URL을 찾을 수 없습니다.")
            return

        # 2. 각 제품별 크롤링
        for rank, url in enumerate(urls, 1):
            print(f"\n[PROCESSING] 제품 [{rank}/{len(urls)}]")

            rows = self.extract_product_and_reviews(
                url, rank, major_category, sub_category
            )

            self.all_data.extend(rows)

            # 중간 저장 (5개마다)
            if rank % 5 == 0:
                self.save_to_excel()
                print(f"  [CHECKPOINT] 중간 저장 완료 (총 {len(self.all_data)}행)")

            time.sleep(2)

        # 최종 저장
        self.save_to_excel()

        elapsed = time.time() - start_time
        print(f"\n{'='*70}")
        print(f"[COMPLETED] 크롤링 완료 (소요시간: {elapsed/60:.1f}분)")
        print(f"[RESULT] 총 {len(self.all_data)}행 수집")
        print(f"{'='*70}")

    def print_stats(self):
        """수집된 데이터 통계 출력"""
        if not self.all_data:
            return

        df = pd.DataFrame(self.all_data)
        print(f"\n{'='*70}")
        print("[STATISTICS] 데이터 통계")
        print(f"{'='*70}")
        print(f"총 데이터 행: {len(df)}")
        print(f"제품 수: {df['product_name'].nunique()}")
        print(f"\n브랜드별 제품 수 (TOP 10):")
        print(df['brand'].value_counts().head(10))

        # 평균 평점 계산
        if 'rating' in df.columns:
            try:
                ratings = df['rating'].replace('', None).dropna()
                if len(ratings) > 0:
                    avg_rating = pd.to_numeric(ratings, errors='coerce').mean()
                    print(f"\n평균 제품 평점: {avg_rating:.2f}")
            except:
                pass

    def close(self):
        """브라우저 종료"""
        self.driver.quit()


def main():
    """
    메인 실행 함수

    사용법:
    - 일본 크롤링: country='Japan'
    - 중국 크롤링: country='China'
    """
    print("\n" + "="*70)
    print("Olive Young Global Web Crawler")
    print("="*70 + "\n")

    # ========================================
    # 여기서 국가 선택 ('Japan','China', 'USA')
    # ========================================
    COUNTRY = 'Japan'  # 국가 입력

    print(f"선택된 국가: {COUNTRY}")
    print("="*70 + "\n")

    crawler = OliveYoungCrawlerAuto(
        country=COUNTRY,
        save_folder='./oliveyoung_data'
    )

    try:
        # 전체 카테고리 크롤링
        crawler.crawl_all_categories()

        # 통계 출력
        crawler.print_stats()

    except KeyboardInterrupt:
        print("\n[INTERRUPTED] 사용자에 의해 중단되었습니다.")
        crawler.save_to_excel()
        crawler.print_stats()
    except Exception as e:
        print(f"\n[ERROR] 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    finally:
        crawler.close()
        print("\n[EXIT] 프로그램 종료")


if __name__ == "__main__":
    main()


Olive Young Global Web Crawler

선택된 국가: Japan

Olive Young Global Crawler - Japan
[INFO] 저장 폴더: c:\Users\cub72\Desktop\공모전\아모퍼\oliveyoung_data

[STEP 0] 국가 선택: 일본
[DEBUG] data-testid로 버튼 찾음
[INFO] 국가 선택 버튼 클릭 중...
[SUCCESS] 국가 선택 팝업 열기 완료
[INFO] 일본 선택 중...
[SUCCESS] 일본 선택 완료
[WARNING] Save 버튼을 찾을 수 없습니다.
[INFO] 엔터를 눌러 진행하거나 수동으로 저장해주세요.

[START] 전체 크롤링 시작 (일본)
[INFO] 총 9개 카테고리
[INFO] 예상 소요 시간: 약 135분

[PROGRESS] 카테고리 [1/9]
[CATEGORY] Skincare > Moisturizers

[CATEGORY] Skincare > Moisturizers

[STEP 1] 카테고리 페이지 접속 중...
[SUCCESS] View 36 설정 완료
[SUCCESS] 제품 URL 수집 완료: 36개

[PROCESSING] 제품 [1/30]
  [PRODUCT  1] アヌア                  | ★2025 アワード★ Anua PDRNヒアルロン酸カプセル100セラム30ml詰め替え限定企画(
               가격: ¥8,176 (할인: 53%) | 평점: 4.8
    [STEP 2] 리뷰 수집 시작...
    [LOADING] More 버튼 클릭 (1/5)
    [LOADING] More 버튼 클릭 (2/5)
    [LOADING] More 버튼 클릭 (3/5)
    [LOADING] More 버튼 클릭 (4/5)
    [SUCCESS] 목표 달성: 50개 리뷰 로드 완료
    [INFO] 발견된 리뷰: 50개
    [PROGRESS] 리뷰 수집 중... 20/50
    [PROGRESS] 리뷰 수집 중...